## Linear Models - Partial Least Squares Regression and the LASSO
#### a.k.a. NBA Player Game Stats vs. Salary

In [ ]:
# load and show dataset
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.cross_decomposition import PLSRegression
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.dummy import DummyRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PowerTransformer, RobustScaler
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
%matplotlib inline
set_matplotlib_formats('svg')
sns.set_style("darkgrid")

In this problem we are dealing with 2 data sets. On the one hand we have the game statistics over the whole season, on the other hand the salary (without bonuses) of NBA players. We want to find out if the salary can be predicted from the game statistics. We want to look at this for the past 2020-21 season.

### 1. Data Inspection and Preprocessing
We use a inner-join. This ensures that we end up with rows of data, which exist in both datasets. Otherwise we could have a players salary but no stats (or vice versa). There is another problem - some players changed the team. We do not respect this fact and take the stats which had been achieved for the team the player had his contract initially.

In [ ]:
data1 = pd.read_csv('salary.csv') #salary stats
data1.drop(columns=['2021-22','2022-23', '2023-24', '2024-25', '2025-26', 'Rk','Guaranteed'], inplace=True) #remove not used columns
data2 = pd.read_csv('stats.csv') #player stats
data2.drop(columns=['Rk'], inplace=True) #remove not used column
data=data1.join(data2.set_index(['Player', 'Tm']), on=['Player', 'Tm'],lsuffix='_caller', rsuffix='_other', how='inner') #inner-join the two datasets
data.reset_index(drop=True, inplace=True)
data.Tm = data.Tm.astype("category").cat.codes #integer coding of teams
data['Signed Using'] = data['Signed Using'].astype("category").cat.codes #integer coding 
data['Pos'] = data['Pos'].astype("category").cat.codes #integer coding
data['2020-21'] = pd.to_numeric(data['2020-21'].map(lambda x: x.lstrip('$'))) #strip dollar sign and convert to numeric

data.head()

We ensured to have "complete" data by using the inner-join on players and team - but there can be still missing data due to other reasons. In a real-world problem, we should now have a look if we can resolve some of the missing values by gathering additional information (match statistics, trading lists and so on). For now, we will do one of the simplest tricks - using the `SimpleImputer` later in our pipelines.

In [ ]:
data.isnull().sum()[(data.isnull().sum() > 0)] #check for missing values

In [ ]:
X = data.drop(columns=['2020-21','Player']) #drop 2020-21 salary (target) and Player
y = data['2020-21']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.25, random_state=21)

You can inspect the correlation matrix below. (Code snippet taken from [seaborn docs](https://seaborn.pydata.org/examples/many_pairwise_correlations.html))

In [ ]:
# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(data.loc[X_train.index].corr(), dtype=bool))
# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))
# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)
# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(data.loc[X_train.index].corr(), mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

### 2. Partial Least Squares Regression

In [ ]:
xscaler = StandardScaler()
yscaler = StandardScaler()
pipe = Pipeline([('imp', SimpleImputer()),('scaler', xscaler), ('plsr', PLSRegression(n_components=3))])
model = TransformedTargetRegressor(regressor=pipe, transformer=yscaler)

In [ ]:
model.fit(X_train,y_train)

We add the X-scores (the "new" latent X-variables) to the dataframe. 

In [ ]:
T = model.regressor_['plsr'].x_scores_
data_pls = pd.DataFrame(columns=X_train.columns,data=xscaler.fit_transform(X_train))
data_pls[['T1', 'T2', 'T3']] = pd.DataFrame(T)
data_pls.head()

Now we observe the scores plot of the first and second component, we integrate the information about the salary by color. We can see that high salaries tend to be in the upper right of the plot. Your task could now also be to index a few player names to the points. But it can be stated that the salary increases with positive T1 and T2. Repeat the plotting and interpretation for the T1-T3 and the T2-T3 level by yourself.

In [ ]:
plt.figure()
sns.scatterplot(x='T1',y='T2',hue=(yscaler.fit_transform(y_train.values.reshape(-1,1))).ravel(), data=data_pls) #change x and y also to T1-T3 and T2-T3
plt.show()

The conterpart to scores plots in PCA/PLSR domain, are the so called loadings plots. Also scatter plots, but now we plot the loadings which are used to transform the oroginal variables to their new latent space. The problem with loading plots is the scaling. Interpretation is easier by using so called correlation loadings. The correlation loadings are bounded between -1 and 1 and thus easier to compare.

In [ ]:
corL = data_pls.corr()
corL.drop(corL.columns[:-3],axis=1,inplace=True)
corL.drop(corL.index[-3:],axis=0,inplace=True)
corL['norm'] = np.linalg.norm(corL, axis=1) #append norm of correlation loadings vectors

Now sort the correlation loadings according to different criteria. For example, by length (norm) or by the highest (absolute) values for different components. You will be able to see that `PTS` 's norm is close to 1 - it seems to be an important predictior.

In [ ]:
sortby = 'norm'
#sortby = 'T1'
corL.sort_values(by=sortby, ascending=False)

Below you can visualize the relationship between a variable one of the three latent variables.

In [ ]:
xvar = 'PTS'
yvar = 'T1'
sns.scatterplot(x=xvar, y=yvar, data=data_pls)

Now we can again go for the correlation loadings plot. We also add the circles of 50% variance and 100% variance.
We can see a very high correlation of the variables `TOV`, `FT`, `FTA`,`FGA`,`FG`,`MP`. The strongest correlation of the original predictors to the 2nd component can be observed for `Age`.  
In T1-T3 space we see that the position of a player seems to be relevant for the 3rd component.  Age is slightly negatively correlated with this component.
The T2-T3 space shows an interesting picture, namely that many features are similarly strongly correlated with the 2nd and 3rd components (note the diagonal).
One can say that the team does not seem to play a role in this model.

In [ ]:
sns.scatterplot(x='T1', y='T2', data=corL)
plt.plot(np.sin(np.linspace(0,2*np.pi,100)),np.cos(np.linspace(0,2*np.pi,100)),'--',color='.5')
plt.plot(np.sqrt(0.5)*np.sin(np.linspace(0,2*np.pi,100)),np.sqrt(.5)*np.cos(np.linspace(0,2*np.pi,100)), '--', color='.8')
for kk,nme in enumerate(corL.index):
     plt.annotate(nme,(corL.T1.values[kk]+.01,corL.T2.values[kk]), fontsize=6)
plt.ylim(-1.1,1.1)
plt.xlim(-1.1,1.1)
plt.figure()
sns.scatterplot(x='T1', y='T3', data=corL)
plt.plot(np.sin(np.linspace(0,2*np.pi,100)),np.cos(np.linspace(0,2*np.pi,100)),'--',color='.5')
plt.plot(np.sqrt(0.5)*np.sin(np.linspace(0,2*np.pi,100)),np.sqrt(.5)*np.cos(np.linspace(0,2*np.pi,100)), '--', color='.8')
for kk,nme in enumerate(corL.index):
     plt.annotate(nme,(corL.T1.values[kk]+.01,corL.T3.values[kk]), fontsize=6)
plt.ylim(-1.1,1.1)
plt.xlim(-1.1,1.1)
plt.show()
plt.figure()
sns.scatterplot(x='T2', y='T3', data=corL)
plt.plot(np.sin(np.linspace(0,2*np.pi,100)),np.cos(np.linspace(0,2*np.pi,100)),'--',color='.5')
plt.plot(np.sqrt(0.5)*np.sin(np.linspace(0,2*np.pi,100)),np.sqrt(.5)*np.cos(np.linspace(0,2*np.pi,100)), '--', color='.8')
for kk,nme in enumerate(corL.index):
     plt.annotate(nme,(corL.T2.values[kk]+.01,corL.T3.values[kk]), fontsize=6)
plt.ylim(-1.1,1.1)
plt.xlim(-1.1,1.1)
plt.show()

So far, we have worked with an initial estimate of the number of compnents to address the specifics of PLS (Scores and loadings plot). In many textbooks only 2 components are often used for visualization and interpretation. However, we will of course perform a grid search to determine the number of components that gives us the smallest RMSE. The nice thing about PLSR is that the parameter space is discrete and constrained.

In [ ]:
cval = KFold(10, random_state=60, shuffle=True)
pipe1 = Pipeline([('imputer', SimpleImputer()),('scaler', ct), ('plsr', PLSRegression(n_components=3))])
model1 = TransformedTargetRegressor(regressor=pipe1, transformer=yscaler)
param_grid1 = dict()
param_grid1['regressor__plsr__n_components'] = np.arange(1,X_train.shape[1])
search1 = GridSearchCV(model1, param_grid1, scoring=['neg_root_mean_squared_error','r2'], n_jobs=-1, cv=cval, refit='neg_root_mean_squared_error', return_train_score=True)

In [ ]:
search1.fit(X_train,y_train)
results1 = pd.DataFrame(search1.cv_results_)
print("Best parameter RMSE=%0.3f):" % (-search1.best_score_))
print(search1.best_params_)

plt.figure()
plt.errorbar(results1.param_regressor__plsr__n_components, -results1.mean_train_neg_root_mean_squared_error, yerr=results1.std_train_neg_root_mean_squared_error, label='Train')
plt.errorbar(results1.param_regressor__plsr__n_components, -results1.mean_test_neg_root_mean_squared_error, yerr=results1.std_test_neg_root_mean_squared_error, label = 'Test')
plt.legend()
plt.xlabel("# of components")
plt.ylabel("RMSE")
plt.title("PLSR")
plt.show()

We can observe a quite high variance for each of the components and we can also see that more components than initially guessed are neccessary for the lowest RMSE in out setting. It is interesting to observe that with 4 coponents we are in a local minimum and close the the best RMSE. The validation error increases, find another (global) minimum at 11 components then we run into overfitting (decreasing RMSE (train) and increasing RMSE (test)). Furthermore we have to state that the RMSE is quite high with approx 5.8 M dollar.

_Remark: Sci-Kit is going for the lowest RMSE, but we could have a more parsimonious model if we would choose 4 components._

### 3. Lasso Regression
As a representative of the regularizing models we choose Lasso. The difference to ridge regression is the norm of the coefficient vector. In this case, the $l_1$ norm is used, which favors a sparse coefficient vector. This implicitly performs a feature selection - non-relevant features are set to 0. However, if there are several strongly correlated (but "important") variables, one may be selected and the others set to zero. The parameter we want to tune via gridsearch is the weight for your constraint for the $l_1$ norm. We already know that we have some strongly correlated features - so we should watch out for the fact that lasso is likely to tends to using only one of those predictors.

In [ ]:
pipe2 = Pipeline([('imputer', SimpleImputer()),('scaler', xscaler), ('lasso', Lasso())])
model2 = TransformedTargetRegressor(regressor=pipe2, transformer=yscaler)
param_grid2 = {'regressor__lasso__alpha': [1e-12,1e-4,1e-3,2e-3, 1e-2,2e-2,2.5e-2, 3e-2,1e-1,2e-1,1, 10, 100]}
search2 = GridSearchCV(model2, param_grid2, scoring=['neg_root_mean_squared_error','r2'], n_jobs=-1, cv=cval, refit='neg_root_mean_squared_error', return_train_score=True)

In [ ]:
search2.fit(X_train, y_train)
print("Best parameter RMSE=%0.3f):" % (-search2.best_score_))
print(search2.best_params_)

plt.figure()
plt.errorbar(param_grid2['regressor__lasso__alpha'],-search2.cv_results_['mean_train_neg_root_mean_squared_error'],yerr=search2.cv_results_['mean_train_neg_root_mean_squared_error'],label='Train')
plt.errorbar(param_grid2['regressor__lasso__alpha'],-search2.cv_results_['mean_test_neg_root_mean_squared_error'],yerr=search2.cv_results_['mean_test_neg_root_mean_squared_error'],label='Test')
plt.gca().set_xscale('log')
plt.legend()
plt.xlabel("alpha")
plt.ylabel("RMSE")
plt.title("Lasso")
plt.show()

We see that moderate regularization leads to the best results. When alpha approaches zero, the model approaches a "normal" linear regression. For larger alpha values, the RMSE deteriorates significantly. The error is in a similar range as for PLSR.

### 4. Final Performance
Finally we will asses the performance on the held out test set.

In [ ]:
best_estimator1 = search1.best_estimator_
y_pred_train1 = best_estimator1.predict(X_train)
y_pred_test1 = best_estimator1.predict(X_test)
best_estimator2 = search2.best_estimator_
y_pred_train2 = best_estimator2.predict(X_train)
y_pred_test2 = best_estimator2.predict(X_test)


minlim = np.min([y_test.min(), np.min(y_pred_test1), np.min(y_pred_test2)])-1e6
maxlim = np.max([y_test.max(), np.max(y_pred_test1), np.max(y_pred_test2)])+1e6
# predicted/actual plot for test set
ax = sns.jointplot(x=y_test.ravel(),y=y_pred_test1.ravel(), height=5,xlim=(minlim,maxlim),ylim=(minlim,maxlim))
ax.ax_joint.set_xlabel('observed player performance')
ax.ax_joint.set_ylabel('predicted player performance')
plt.show()


# predicted/actual plot for test set
ax = sns.jointplot(x=y_test.ravel(),y=y_pred_test2.ravel(), height=5,xlim=(minlim,maxlim),ylim=(minlim,maxlim))
ax.ax_joint.set_xlabel('observed player performance')
ax.ax_joint.set_ylabel('predicted player performance')
plt.show()

In [ ]:
### lr baseline
pipelr = Pipeline([('imputer', SimpleImputer()),('scaler', xscaler), ('lr', LinearRegression())])
modellr = TransformedTargetRegressor(regressor=pipelr, transformer=yscaler)
modellr.fit(X_train,y_train)

### dummy worst case
pipedm = Pipeline([('imputer', SimpleImputer()),('scaler', xscaler), ('dm', DummyRegressor())])
modeldm = TransformedTargetRegressor(regressor=pipedm, transformer=yscaler)
modeldm.fit(X_train,y_train)


est = {'PLSR': best_estimator1, 'Lasso': best_estimator2, 'LinReg': modellr, 'Dummy': modeldm}

fin_res = pd.DataFrame({'model': ['PLSR', 'Lasso','LinReg', 'Dummy'], 'RMSE train': np.empty(4), 'R2 train': np.empty(4), 'RMSE test': np.empty(4), 'R2 test': np.empty(4)}).set_index('model')

for m in ['PLSR', 'Lasso', 'LinReg','Dummy']:
    y_pred_train = est[m].predict(X_train)
    y_pred_test = est[m].predict(X_test)
    fin_res.at[m,'RMSE train'] = np.sqrt(mean_squared_error(y_train,y_pred_train))
    fin_res.at[m,'R2 train'] = r2_score(y_train,y_pred_train)
    fin_res.at[m,'RMSE test'] = np.sqrt(mean_squared_error(y_test,y_pred_test))
    fin_res.at[m,'R2 test'] = r2_score(y_test,y_pred_test)
fin_res

Interestingly the linear regression works best. 

In [ ]:
f,axs = plt.subplots(1,3, figsize=(12,4))
coef1 = pd.Series(best_estimator1.regressor_['plsr'].coef_.ravel(), index = X_train.columns).sort_values(key=lambda x: np.abs(x))
coef2 = pd.Series(best_estimator2.regressor_['lasso'].coef_, index = X_train.columns).sort_values(key=lambda x: np.abs(x))
coef3 = pd.Series(modellr.regressor_['lr'].coef_, index = X_train.columns).sort_values(key=lambda x: np.abs(x))

coef1.plot(kind = "barh",ax = axs[0])
axs[0].set_title("Coefficients PLSR")
axs[0].set_xlim(-.4,1)

coef2.plot(kind = "barh",ax = axs[1])
axs[1].set_title("Coefficients Lasso")
axs[1].set_xlim(-.4,1)

coef3.plot(kind = "barh",ax = axs[2])
axs[2].set_title("Coefficients LinReg")
axs[2].set_xlim(-.4,1)

plt.tight_layout()

We can see that Lasso sets many coefficients to zero - especially compared with the linear regression.